### Multi Classification 
: 다중분류  
활성화 함수(Activation function) 으로 softmax함수가 사용된다


In [1]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(5)

In [2]:
# 데이터 가져오기
xy =np.loadtxt('data-04-zoo.csv',delimiter=',',dtype=np.float32)
print(xy.shape)  # (101,17)

# 학습 데이터 : 70% , 70개
x_train = xy[:70,:-1]
y_train = xy[:70,[-1]]
print(x_train.shape,y_train.shape)

# 검증 데이터 : 30%, 31개
x_test = xy[70:,:-1]
y_test = xy[70:,[-1]]
print(x_test.shape,y_test.shape)

(101, 17)
(70, 16) (70, 1)
(31, 16) (31, 1)


In [3]:
# one-hot 인코딩
# Category : 0 ~ 6 (7개)
# 0 : [1 0 0 0 0 0 0]
# 1 : [0 1 0 0 0 0 0]
# 2 : [0 0 1 0 0 0 0]
# 3 : [0 0 0 1 0 0 0]
# ...
# 6 : [0 0 0 0 0 0 1]
nb_classes = 7   # 분류(class) 갯수(0,1,2,3,4,5,6)
Y_one_hot = tf.one_hot(y_train,nb_classes)     # [None,1,7]
print(Y_one_hot.shape)                         # [70,1,7]  , Rank=3 (3차원)
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot.shape)                         # [70,7]    , Rank=2 (2차원)
# Y_one_hot

(70, 1, 7)
(70, 7)


In [4]:
# 변수 초기화 : weight, bias
# (m,n) * (n,l)  = (m,l) : 행렬의 내적 곱셈 공식
# (70,16) * (16,7) = (70,7)
#  X    *   W     =   Y
W = tf.Variable(tf.random.normal([16,nb_classes]),name='weight')
b = tf.Variable(tf.random.normal([nb_classes]),name='bias')

In [5]:
# 예측 함수(hypothesis) : H(X) = softmax(W*X + b)
def logits(X):
    return tf.matmul(X,W) + b

def hypothesis(X):
    return tf.nn.softmax(logits(X))  

In [6]:
# 비용함수 구현 방법 2 : tf.nn.softmax_cross_entropy_with_logits() 함수 사용
def cost_func():
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits(x_train),
                                             labels = Y_one_hot)
    cost =  tf.reduce_mean(cost_i)
    return cost

In [7]:
# 경사 하강법
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [8]:
# 학습 시작
print('***** Start Learning!!')
for step in range(5001):
    optimizer.minimize(cost_func,var_list=[W,b])

    if step % 1000 == 0:
        print('%04d'%step,'cost:[',cost_func().numpy(),']')            
    
print('***** Learning Finished!!')     

***** Start Learning!!
0000 cost:[ 4.3771977 ]
1000 cost:[ 0.0036812557 ]
2000 cost:[ 0.0011356688 ]
3000 cost:[ 0.0005179679 ]
4000 cost:[ 0.0002727554 ]
5000 cost:[ 0.00015376032 ]
***** Learning Finished!!


In [9]:
# 회귀 계수, weight과 bias 출력
# print('Weight:',W.numpy())
# print('Bias:',b.numpy())

In [23]:
# 정확도 측정 : accuracy computation
Y_one_hot = tf.one_hot(y_test,nb_classes)      # [None,1,7]
print(Y_one_hot.shape)                         # [31,1,7]  , Rank=3 (3차원)
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes])
print(Y_one_hot.shape)                         # [31,7]    , Rank=2 (2차원)
# Y_one_hot

# tf.argmax() : 값이 가장 큰 요소의 인덱스 값을 반환
def predict(X):
    return tf.argmax(hypothesis(X),axis=1)

correct_predict = tf.equal(predict(x_test),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))
print("Accuracy:",accuracy.numpy()) # Accuracy: 0.8064516

(31, 1, 7)
(31, 7)
Accuracy: 0.8064516


In [24]:
#예측
print('***** Predict')
pred = predict(x_test).numpy()
for p,y in zip(pred, y_test.flatten()):
    print("[{}] Prediction: {} / Real Y: {}".format(p == int(y), p, int(y)))

***** Predict
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 1 / Real Y: 1
[False] Prediction: 5 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 0 / Real Y: 0
[False] Prediction: 4 / Real Y: 2
[True] Prediction: 6 / Real Y: 6
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 2 / Real Y: 2
[False] Prediction: 2 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 6 / Real Y: 6
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 5 / Real Y: 5
[True] Prediction: 4 / Real Y: 4
[False] Prediction: 0 / Real Y: 2
[False] Prediction: 4 / Real Y: 2
[True] Prediction: 3 / Real Y: 3
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 1 / Real Y: 1
[True] Prediction: 0 / Real Y: 0
[True] Prediction: 5 / Real Y: 5
[True] Prediction: 0 / Real Y: 0
[False] Prediction: 2 / 